# Example 2: MPC

In this example, we will formulate a simple MPC controller using Pyomo.

We consider a simple (one-state, SISO) linear system with state $x_k$ and an input $u_k$.
- The prediction horizon is $N = 4$, with system dynamics given by $a = 0.8104$ and $b = 0.2076$.
- The cost function imposes a quadratic penalty on state and control signals, leading to an optimization problem:
\begin{align}
    \min \quad & \sum_{k=0}^{N-1} x_{k+1}^2 + u_k^2 \\
    \text{subject to:} \quad & x_{k+1} = a x_k + b u_k, \quad k = 0, \ldots,N-1 \\
    & x_0 = x_\text{init}
\end{align}
- The initial state (the present state measurement) is unknown when we specify the optimization problem. As a concrete example, we will solve for $x_\text{init} = 0.4884$.

Since we don't know the initial state, we use the concept of an abstract model in Pyomo:

In [6]:
from pyomo.environ import *

# create model
m = AbstractModel()

For illustration purposes, we will assume that all parameters ($N$, $a$ and $b$ in addition to $x_\text{init}$) are unknown:

In [7]:
# Parameters
m.a = Param() # by default, a real-valued parameter
m.b = Param()
m.N = Param(domain=NonNegativeIntegers) # N is a positive integer
m.xinit = Param()

Then, we define variables, states and inputs (vectors of length $N+1$ and $N$ -- we let the state vector be length $N+1$ since we want to include the initial state in this vector, for simplicity).

In [8]:
m.K = RangeSet(1, m.N)
m.Kp = RangeSet(1, m.N+1)
m.x = Var(m.Kp) # one step longer since it will also contain x[0]
m.u = Var(m.K)

When defining the objective function, we cannot construct an expression for the objective since we don't know the value of N (or the other parameters). We must instead define a 'rule' (a function) for the objective:

In [9]:
def objective_rule(m):
    return sum(m.x[k+1]*m.x[k+1]+ m.u[k]*m.u[k] for k in m.K)

m.obj = Objective( rule = objective_rule )

The same for the constraints:

In [10]:
def model_constraint(m,k):
    return m.x[k+1] == m.a*m.x[k] + m.b*m.u[k]

def initial_constraint(m):
    return m.x[1] == m.xinit

m.initconstraint = Constraint( rule = initial_constraint )
m.modelconstraint = Constraint( m.K, rule = model_constraint )

The above specifies the model (of the optimization problem), and we can proceed to solving it. Here, we outline two different methods. 
- The first method is within the Python script. The optimization model must then be instantiated with values assigned to parameters, before we can solve it as before (using now ipopt since the model is a QP).
- The second method uses the Python script defining the model (the Python lines above) as input argument to a command line call to Pyomo. The other input arguments are a file defining the parameters, and specifying the solver.

Method 1 (within Python):

In [11]:
# Set parameters and instantiate model
m.a = 0.8104
m.b = 0.2076
m.N = 4
m.xinit = 0.4884

m_instance = m.create_instance()

# Solve QP using ipopt
from pyomo.opt import SolverFactory
opt = SolverFactory('ipopt')
results = opt.solve(m_instance)

results.write()

# ==========================================================
# = Solver Results                                         =
# ==========================================================
# ----------------------------------------------------------
#   Problem Information
# ----------------------------------------------------------
Problem: 
- Lower bound: -inf
  Upper bound: inf
  Number of objectives: 1
  Number of constraints: 5
  Number of variables: 9
  Sense: unknown
# ----------------------------------------------------------
#   Solver Information
# ----------------------------------------------------------
Solver: 
- Status: ok
  Message: Ipopt 3.11.1\x3a Optimal Solution Found
  Termination condition: optimal
  Id: 0
  Error rc: 0
  Time: 0.08776593208312988
# ----------------------------------------------------------
#   Solution Information
# ----------------------------------------------------------
Solution: 
- number of solutions: 0
  number of solutions displayed: 0


In the second method, instead of specifying the values of the parameters in the Python code, we assume they are given in a file (with the AMPL format). This file looks as below (a line starting with hashtag is a comment, each line is ended by semicolon).

In [3]:
!type Example2.dat

# Data for Example 2 (MPC) in AMPL format

param a := 0.8104 ;
param b := 0.2076 ;
param N := 4 ;
param xinit := 0.4884 ;


We can now invoke Pyomo from the command line: